**安裝Pytorch**

In [ ]:
# Install Pytorch
!pip install torch torchvision

In [ ]:
# Check Pytoch
import torch
print(torch.__version__)
torch.cuda.is_available()

1.5.0+cu101


True

**連接Google Driver**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!ls /content/drive/My\ Drive/Colab\ Notebooks/Bert_MovieReview

BertMovieReview.ipynb  submit.csv  trained_model
sampleSubmission.csv   test.tsv    train.tsv


**Install Transformers**

In [ ]:
!pip install transformers

     |████████████████████████████████| 573kB 15.9MB/s 
     |████████████████████████████████| 3.7MB 55.3MB/s 
     |████████████████████████████████| 1.0MB 59.5MB/s 
     |████████████████████████████████| 890kB 48.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=31f8ff85eff74ac357c2d46e99313f5f3100a3625a8649c148653980fe8625d8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import os
import pickle
import torch
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F # 激勵函數

In [ ]:
from transformers import BertTokenizer
# 使用English的 Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')


In [ ]:
vocab = tokenizer.vocab
print("字典大小:",len(vocab))

字典大小: 30522


In [ ]:
# Tokenize Example 節錄 from newyork times.
tokensForScentence = tokenizer.tokenize('There have been 68 recent deaths of residents and nurses from the facility in a small New Jersey town.')
tokenizer.convert_tokens_to_ids(tokensForScentence)

[2045,
 2031,
 2042,
 6273,
 3522,
 6677,
 1997,
 3901,
 1998,
 11500,
 2013,
 1996,
 4322,
 1999,
 1037,
 2235,
 2047,
 3933,
 2237,
 1012]

**Use Bert**

In [ ]:
cd /content/drive/My\ Drive/Colab\ Notebooks/Bert_MovieReview

/content/drive/My Drive/Colab Notebooks/Bert_MovieReview


In [ ]:
!ls

BertMovieReview.ipynb  submit.csv  trained_model
sampleSubmission.csv   test.tsv    train.tsv


In [ ]:
import pandas as pd

In [ ]:
trainPd = pd.read_csv('train.tsv', sep='\t', header=0)
trainPd.head(5)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [ ]:
# 打亂數據 
trainPd=trainPd.sample(frac=1.0)
# 打亂後重新給予 Index 
trainPd=trainPd.reset_index(drop=True)

In [ ]:
trainPd.head(5)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [ ]:
# train Data 總共有 156060
# 分成 trainingSet : 109242 / testingSet : ~

mTrainingSet = trainPd.iloc[:109242]
mTestingSet = trainPd.iloc[109243:]


In [ ]:
# 將答案轉換成數字，為了做分類時可以比較好進行訓練
class DataDic(object):
    def __init__(self, answers):
        self.answers = answers # 全部答案(含重複)
        self.answers_norepeat = sorted(list(set(answers))) # 不重複
        self.answers_types = len(self.answers_norepeat) # 總共多少類
        self.ans_list = [] # 用於查找id或是text的list
        self._make_dic() # 製作字典

    def _make_dic(self):
        for index_a,a in enumerate(self.answers_norepeat):
            if a != None:
                self.ans_list.append((index_a,a))

    def to_id(self,text):
        for ans_id,ans_text in self.ans_list:
            if text == ans_text:
                return ans_id

    def to_text(self,id):
        for ans_id,ans_text in self.ans_list:
            if id == ans_id:
                return ans_text
    @property
    def types(self):
        return self.answers_types
    
    @property
    def data(self):
        return self.answers

    def __len__(self):
        return len(self.answers)

In [ ]:
tmp = mTrainingSet.head(5)

for index,row in tmp.iterrows():
  print("段落字串:",row['Phrase'])
  print("情緒:",row['Sentiment'])

tmp = mTestingSet.head(5)

for index,row in tmp.iterrows():
  print("段落字串:",row['Phrase'])
  print("情緒:",row['Sentiment'])

段落字串: A journey spanning nearly three decades of bittersweet camaraderie and history , in which we feel that we truly know what makes Holly and Marina tick , and our hearts go out to them as both continue to negotiate their imperfect , love-hate relationship .
情緒: 3
段落字串: several cliched movie structures
情緒: 1
段落字串: of a determined woman 's courage to find her husband in a war zone
情緒: 3
段落字串: morality , family ,
情緒: 2
段落字串: both hats
情緒: 2
段落字串: does n't want to
情緒: 1
段落字串: works as well as it does because of the performances .
情緒: 2
段落字串: all places
情緒: 2
段落字串: wickedly fun
情緒: 4
段落字串: the payoff for the audience , as well as the characters , is messy , murky , unsatisfying .
情緒: 0


In [ ]:
# 轉換 Data to Feature
def DataToFeature( dataSetPd ) :
  tmpQuestions = []
  tmpAnswers = []

  for index,row in dataSetPd.iterrows():
    try:
      tmpQuestions.append(row['Phrase'])
      tmpAnswers.append(row['Sentiment'])
    except:
      continue

  question_dic = DataDic(tmpQuestions)
  ans_dic = DataDic(tmpAnswers)
  return ( question_dic , ans_dic )


In [ ]:
# 處理資料成Feautre
train_question_dic , train_ans_dic = DataToFeature(mTrainingSet)
test_question_dic , test_ans_dic = DataToFeature(mTestingSet)

In [ ]:
print("全部答案:",len(test_ans_dic))
print("全部答案種類:",test_ans_dic.types)
a_id = test_ans_dic.to_id(1) # 將句子轉換成id
a_text = test_ans_dic.to_text(1) # 將id轉換成句子
print("測試轉換:", a_id, a_text)


全部答案: 46817
全部答案種類: 5
測試轉換: 1 1


In [ ]:
count = 0
for ele in train_ans_dic.ans_list :
  if count > 5 :
    break
  count = count + 1 
  print(ele)

(0, 0)
(1, 1)
(2, 2)
(3, 3)
(4, 4)


In [ ]:
max_seq_len = 0

In [ ]:
train_q_tokens = []
max_seq_len = 0
for q in train_question_dic.data:
  bert_ids = tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(q)))
  if(len(bert_ids)>max_seq_len):
    max_seq_len = len(bert_ids)
  train_q_tokens.append(bert_ids)

print("最長問句長度For Traing:",max_seq_len)

最長問句長度For Traing: 78


In [ ]:
test_q_tokens = []
for q in test_question_dic.data:
  bert_ids = tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(q)))
  if(len(bert_ids)>max_seq_len):
    max_seq_len = len(bert_ids)
  test_q_tokens.append(bert_ids)

print("最長問句長度For Testing:",max_seq_len)

最長問句長度For Testing: 80


In [ ]:
# 全部資料最長句子長度 : 80
max_seq_len = 80 

In [ ]:
print(train_q_tokens[0])
# [CLS] XXXXX [SEP]
# tokenizer.convert_ids_to_tokens([101, 8271, 5637, 1266, 5971, 6123, 5059, 9065, 102])
# '[CLS]', '2018', '臺', '北', '藝', '術', '節', 'faq', '[SEP]'

[101, 1037, 4990, 13912, 3053, 2093, 5109, 1997, 8618, 26760, 15558, 11503, 5400, 4063, 2666, 1998, 2381, 1010, 1999, 2029, 2057, 2514, 2008, 2057, 5621, 2113, 2054, 3084, 9079, 1998, 9985, 16356, 1010, 1998, 2256, 8072, 2175, 2041, 2000, 2068, 2004, 2119, 3613, 2000, 13676, 2037, 29238, 1010, 2293, 1011, 5223, 3276, 1012, 102]


In [ ]:
tokenizer.convert_ids_to_tokens([101, 1037, 4990, 13912, 3053, 2093, 5109, 1997, 8618, 26760, 15558, 11503, 5400, 4063, 2666, 1998, 2381, 1010, 1999, 2029, 2057, 2514, 2008, 2057, 5621, 2113, 2054, 3084, 9079, 1998, 9985, 16356, 1010, 1998, 2256, 8072, 2175, 2041, 2000, 2068, 2004, 2119, 3613, 2000, 13676, 2037, 29238, 1010, 2293, 1011, 5223, 3276, 1012, 102])

['[CLS]',
 'a',
 'journey',
 'spanning',
 'nearly',
 'three',
 'decades',
 'of',
 'bitter',
 '##sw',
 '##eet',
 'cam',
 '##ara',
 '##der',
 '##ie',
 'and',
 'history',
 ',',
 'in',
 'which',
 'we',
 'feel',
 'that',
 'we',
 'truly',
 'know',
 'what',
 'makes',
 'holly',
 'and',
 'marina',
 'tick',
 ',',
 'and',
 'our',
 'hearts',
 'go',
 'out',
 'to',
 'them',
 'as',
 'both',
 'continue',
 'to',
 'negotiate',
 'their',
 'imperfect',
 ',',
 'love',
 '-',
 'hate',
 'relationship',
 '.',
 '[SEP]']

In [ ]:
# 補齊長度 train_q_tokens
original_length_train = []
for q in train_q_tokens:
  length = len(q)
  original_length_train.append(length)
  while len(q)<max_seq_len:
    q.append(0)

# 補齊長度 test_q_tokens
original_length_test = []
for q in test_q_tokens:
  length = len(q)
  original_length_test.append(length)
  while len(q)<max_seq_len:
    q.append(0)

In [ ]:
train_a_labels = []
for a in train_ans_dic.data:
  train_a_labels.append(train_ans_dic.to_id(a))

test_a_labels = []
for a in test_ans_dic.data:
  test_a_labels.append(test_ans_dic.to_id(a))

In [ ]:
# BERT input embedding - training
input_ids = train_q_tokens
input_segment_ids = [[0]*max_seq_len for i in range(len(train_question_dic))]         # token_type_ids # segment_ids儲存的是句子的id，id為0就是第一句，id為1就是第二句
input_masks = []                                                                # position_ids:1代表是真實的單詞id，0代表補齊長度
for i in range(len(train_question_dic)):
    position_ids = []
    for j in range(original_length_train[i]):
        position_ids.append(1)
    while len(position_ids)<max_seq_len:
        position_ids.append(0)
    input_masks.append(position_ids)
answer_lables = train_a_labels
assert len(input_ids) == len(train_question_dic) and len(input_ids) == len(input_masks) and len(input_ids) == len(input_segment_ids)
qaData_features = {'input_ids':input_ids,
                'input_segment_ids':input_segment_ids,
                'input_masks':input_masks,
                'answer_lables':answer_lables,
                'question_dic':train_question_dic,
                'answer_dic':train_ans_dic}

# 因為train_data保證可以拿到所有類別的資料，且之後要做predict時才能將預測的結果做正確轉換
output = open('trained_model/data_features.pkl', 'wb')
pickle.dump(qaData_features, output)


In [ ]:
# 查看 資料
print(qaData_features['input_ids'][2])
print(tokenizer.convert_ids_to_tokens(qaData_features['input_ids'][2]))
print(qaData_features['input_segment_ids'][2])
print(qaData_features['input_masks'][2])
print(qaData_features['answer_lables'][2])

[101, 1997, 1037, 4340, 2450, 1005, 1055, 8424, 2000, 2424, 2014, 3129, 1999, 1037, 2162, 4224, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['[CLS]', 'of', 'a', 'determined', 'woman', "'", 's', 'courage', 'to', 'find', 'her', 'husband', 'in', 'a', 'war', 'zone', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[0, 0, 0, 0

In [ ]:
# BERT input embedding - testing
input_ids = test_q_tokens
input_segment_ids = [[0]*max_seq_len for i in range(len(test_question_dic))]         # token_type_ids # segment_ids儲存的是句子的id，id為0就是第一句，id為1就是第二句
input_masks = []                                                                # position_ids:1代表是真實的單詞id，0代表補齊長度
for i in range(len(test_question_dic)):
    position_ids = []
    for j in range(original_length_test[i]):
        position_ids.append(1)
    while len(position_ids)<max_seq_len:
        position_ids.append(0)
    input_masks.append(position_ids)
answer_lables = test_a_labels
assert len(input_ids) == len(test_question_dic) and len(input_ids) == len(input_masks) and len(input_ids) == len(input_segment_ids)
testQaData_features = {'input_ids':input_ids,
                'input_segment_ids':input_segment_ids,
                'input_masks':input_masks,
                'answer_lables':answer_lables,
                'question_dic':test_question_dic,
                'answer_dic':test_ans_dic}


In [ ]:
# 查看 資料
print(testQaData_features['input_ids'][2])
print(tokenizer.convert_ids_to_tokens(testQaData_features['input_ids'][2]))
print(testQaData_features['input_segment_ids'][2])
print(testQaData_features['input_masks'][2])
print(testQaData_features['answer_lables'][2])
print(len(testQaData_features['input_ids']))
print(len(testQaData_features['input_segment_ids']))
print(len(testQaData_features['input_masks']))
print(len(testQaData_features['answer_lables']))

[101, 2035, 3182, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['[CLS]', 'all', 'places', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
def makeDataset(data_feature):
    input_ids = data_feature['input_ids']
    input_segment_ids = data_feature['input_segment_ids']
    input_masks = data_feature['input_masks']
    answer_lables = data_feature['answer_lables']

    all_input_ids = torch.tensor([input_id for input_id in input_ids], dtype=torch.long)
    all_input_segment_ids = torch.tensor([input_segment_id for input_segment_id in input_segment_ids], dtype=torch.long)
    all_input_masks = torch.tensor([input_mask for input_mask in input_masks], dtype=torch.long)
    all_answer_lables = torch.tensor([answer_lable for answer_lable in answer_lables], dtype=torch.long)
    dataset = TensorDataset(all_input_ids, all_input_segment_ids, all_input_masks, all_answer_lables)

    return dataset

**Fine-Tuning**

In [ ]:
def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

In [ ]:
bert_config, bert_class, bert_tokenizer = (BertConfig, BertForSequenceClassification, BertTokenizer)

# Set use gpu
device = torch.device("cuda")

train_dataset = makeDataset(qaData_features)
test_dataset = makeDataset(testQaData_features)
train_dataloader = DataLoader(train_dataset ,batch_size=8 ,shuffle=True)
test_dataloader = DataLoader(test_dataset ,batch_size=8 ,shuffle=True)

config = bert_config.from_pretrained('bert-base-uncased',num_labels = 5)
model = bert_class.from_pretrained('bert-base-uncased', from_tf=bool('.ckpt' in 'bert-base-uncased'), config=config)
model.to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
Learning_rate = 5e-6       # 學習率
optimizer = AdamW(optimizer_grouped_parameters, lr=Learning_rate, eps=1e-8)

for epoch in range(2):
    # 訓練模式
    model.train()
    All_train_correct = 0.0
    AllTrainLoss = 0.0
    count = 0
    for batch_dict in train_dataloader:
        batch_dict = tuple(t.to(device) for t in batch_dict)
        
        # From makeDataset : TensorDataset(all_input_ids, all_input_segment_ids, all_input_masks, all_answer_lables)
        # [0] : id
        # [1] : segment id
        # [2] : mask
        # [3] : answer label

        outputs = model(
            batch_dict[0],
            token_type_ids = batch_dict[1],
            attention_mask = batch_dict[2],
            labels = batch_dict[3]
            )
        loss, logits = outputs[:2]
        
        train_correct = compute_accuracy(logits, batch_dict[3])       # 計算正確率
        All_train_correct += train_correct
        AllTrainLoss += loss.item()
        count += 1

        model.zero_grad()
        loss.backward()
        optimizer.step()
        
    Average_train_correct = round(All_train_correct/count, 3)
    Average_train_loss = round(AllTrainLoss/count, 3)

    # 測試模式
    model.eval()
    All_test_correct = 0.0
    AllTestLoss = 0.0
    count = 0
    for batch_dict in test_dataloader:
        batch_dict = tuple(t.to(device) for t in batch_dict)

        outputs = model(
            batch_dict[0],
            token_type_ids = batch_dict[1],
            attention_mask = batch_dict[2],
            labels = batch_dict[3]
            )
        loss, logits = outputs[:2]

        test_correct = compute_accuracy(logits, batch_dict[3])       # 計算正確率
        All_test_correct += test_correct
        AllTestLoss += loss.item()

        count += 1
    
    Average_test_correct = round(All_test_correct/count, 3)
    Average_test_loss = round(AllTestLoss/count, 3)

    print('第' + str(epoch+1) + '次' + '訓練模式，loss為:' + str(Average_train_loss) + ' 正確率為' + str(Average_train_correct)+ '，測試模式，loss為:' + str(Average_test_loss) + ' 正確率為' + str(Average_test_correct))

model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained('trained_model')


第1次訓練模式，loss為:0.885 正確率為65.768，測試模式，loss為:0.729 正確率為69.21
第2次訓練模式，loss為:0.686 正確率為71.598，測試模式，loss為:0.713 正確率為69.9


In [ ]:
def toBertIds(question_input):
    return tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(question_input)))

In [ ]:
print( toBertIds('intermittently pleasing but mostly routine effort') )

[101, 23852, 2135, 24820, 2021, 3262, 9410, 3947, 102]


In [ ]:
# Handler Test.tsv
testPd = pd.read_csv('test.tsv', sep='\t', header=0)
testPd.head(5)

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [ ]:
def DataToFeatureV2( dataSetPd ) :
  tmpQuestions = []
  tmpAnswers = []

  for index,row in dataSetPd.iterrows():
    try:
      tmpQuestions.append(row['Phrase'])
      tmpAnswers.append(0)
    except:
      continue

  question_dic = DataDic(tmpQuestions)
  ans_dic = DataDic(tmpAnswers)
  return ( question_dic , ans_dic )

In [ ]:
submit_question_dic , submit_ans_dic = DataToFeatureV2(testPd)

In [ ]:
print(submit_question_dic.data[0])

An intermittently pleasing but mostly routine effort .


In [ ]:
predictQs = []
for q in submit_question_dic.data : 
  bert_ids = toBertIds(q)
  assert len(bert_ids) <= 512
  # 補齊長度
  while len(bert_ids)<512:
    bert_ids.append(0)
  predictQs.append(bert_ids)

print(predictQs[2])

[101, 2019, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
def makeDataset(data_feature):
    input_ids = data_feature['input_ids']
    input_segment_ids = data_feature['input_segment_ids']
    input_masks = data_feature['input_masks']
    answer_lables = data_feature['answer_lables']

    all_input_ids = torch.tensor([input_id for input_id in input_ids], dtype=torch.long)
    all_input_segment_ids = torch.tensor([input_segment_id for input_segment_id in input_segment_ids], dtype=torch.long)
    all_input_masks = torch.tensor([input_mask for input_mask in input_masks], dtype=torch.long)
    all_answer_lables = torch.tensor([answer_lable for answer_lable in answer_lables], dtype=torch.long)
    dataset = TensorDataset(all_input_ids, all_input_segment_ids, all_input_masks, all_answer_lables)

    return dataset

In [ ]:
# Load Model And Init

pkl_file = open('trained_model/data_features.pkl', 'rb')
data_features = pickle.load(pkl_file)
pkl_file.close()
answer_dic = data_features['answer_dic']

print( answer_dic.data )

[3, 1, 3, 2, 2, 1, 2, 4, 2, 2, 3, 2, 3, 3, 0, 1, 2, 2, 3, 1, 3, 2, 2, 3, 3, 1, 3, 2, 2, 3, 4, 3, 1, 3, 2, 2, 3, 3, 3, 2, 2, 2, 3, 4, 3, 1, 2, 3, 2, 2, 2, 2, 3, 2, 2, 0, 2, 2, 2, 1, 1, 0, 2, 2, 3, 2, 2, 3, 0, 3, 3, 1, 2, 2, 2, 3, 3, 2, 2, 1, 2, 2, 3, 3, 2, 2, 1, 1, 2, 3, 3, 1, 2, 2, 3, 3, 2, 2, 2, 2, 3, 1, 3, 0, 4, 1, 2, 1, 1, 2, 2, 3, 2, 1, 3, 4, 1, 2, 1, 2, 2, 2, 2, 3, 2, 0, 2, 2, 1, 1, 3, 2, 2, 3, 1, 0, 2, 2, 2, 2, 2, 3, 1, 0, 2, 3, 1, 3, 2, 4, 3, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 3, 4, 4, 3, 2, 2, 1, 2, 2, 2, 3, 1, 2, 0, 1, 3, 2, 2, 2, 1, 4, 4, 3, 4, 1, 3, 1, 4, 2, 2, 2, 4, 2, 2, 3, 2, 0, 1, 1, 3, 1, 2, 3, 2, 1, 1, 3, 1, 1, 2, 2, 2, 3, 3, 2, 2, 2, 1, 3, 3, 4, 3, 3, 4, 2, 0, 2, 3, 2, 2, 1, 3, 4, 2, 2, 4, 2, 2, 4, 2, 3, 2, 2, 2, 4, 2, 1, 4, 3, 0, 1, 2, 2, 2, 1, 2, 2, 0, 4, 3, 3, 0, 2, 2, 2, 3, 2, 2, 3, 1, 2, 2, 2, 2, 3, 4, 2, 3, 2, 3, 2, 1, 2, 2, 4, 2, 2, 2, 3, 2, 3, 1, 1, 2, 1, 2, 2, 2, 2, 2, 3, 2, 3, 3, 0, 2, 2, 1, 3, 2, 2, 2, 1, 2, 1, 4, 3, 2, 2, 2, 3, 3, 3, 2, 4, 2, 2, 3, 2, 1, 1, 

In [ ]:
print( answer_dic.answers_types )

5


In [ ]:
bert_config, bert_class, bert_tokenizer = (BertConfig, BertForSequenceClassification, BertTokenizer)
# set device
device = torch.device("cuda")
bert_config, bert_class, bert_tokenizer = (BertConfig, BertForSequenceClassification, BertTokenizer)
config = bert_config.from_pretrained('trained_model/config.json')
model = bert_class.from_pretrained('trained_model/pytorch_model.bin', from_tf=bool('.ckpt' in 'bert-base-uncased'), config=config)
model.to(device)
model.eval()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# load and init
pkl_file = open('trained_model/data_features.pkl', 'rb')
data_features = pickle.load(pkl_file)
pkl_file.close()
answer_dic = data_features['answer_dic']

bert_config, bert_class, bert_tokenizer = (BertConfig, BertForSequenceClassification, BertTokenizer)
config = bert_config.from_pretrained('trained_model/config.json')
model = bert_class.from_pretrained('trained_model/pytorch_model.bin', from_tf=bool('.ckpt' in 'bert-base-uncased'), config=config)
model.eval()


In [ ]:
# 預測測試
# 段落字串: spiritual journey 情緒: 3
# 段落字串: will have an opinion to share 情緒: 2
# 段落字串: so downbeat and 情緒: 1

mq = questions[50000] #'spiritual journey'
bert_ids = toBertIds(mq)
assert len(bert_ids) <= 512
# 補齊長度
while len(bert_ids)<512:
  bert_ids.append(0)
input_ids = torch.LongTensor(bert_ids).unsqueeze(0)

# predict時，因為沒有label所以沒有loss
outputs = model(input_ids)
predicts = outputs[0]
max_val = torch.max(predicts)
label = (predicts == max_val).nonzero().numpy()[0][1] # 在第1維度取最大值並返回索引值 

ans_label = answer_dic.to_text(label)

print(label)

2


In [ ]:
questions = []
questionsId = []
finalAnswers = []
finalQuestionsId = []
for index,row in testPd.iterrows():
  questionsId.append(row['PhraseId'])
  questions.append(row['Phrase'])

In [ ]:
print(len(questions))
print(len(questionsId))
print(len(finalAnswers))
print(len(finalQuestionsId))

66292
66292
0
0


In [ ]:
count = 0;
for index, question in enumerate(questions):
    #if count<10 :
    #  count = count + 1 
    #  continue
    bert_ids = toBertIds(question)
    assert len(bert_ids) <= 512
    # 補齊長度
    while len(bert_ids)<512:
        bert_ids.append(0)
    input_ids = torch.LongTensor(bert_ids).unsqueeze(0)
    
    # predict時，因為沒有label所以沒有loss
    outputs = model(input_ids)

    predicts = outputs[0]
    max_val = torch.max(predicts)
    label = (predicts == max_val).nonzero().numpy()[0][1] # 在第1維度取最大值並返回索引值 
    ans_label = answer_dic.to_text(label)
    finalAnswers.append(ans_label);
    finalQuestionsId.append(questionsId[count])
    if( count%1000 == 0 ) :
      print("第" + str(count) +  "題,Ans:" + str(ans_label) )    
    count = count + 1 
    if count>=10000 :
      break;

print("總共" + str(len(questions)) + "題")
# Write CSV test
dataframeWriter = pd.DataFrame({'PhraseId':finalQuestionsId,'Ans':finalAnswers})
#將DataFrame儲存為csv,index表示是否顯示行名，default=True
dataframeWriter.to_csv("submit.csv",index=True,sep=',')

第0題,Ans:2
第1000題,Ans:2
第2000題,Ans:2
第3000題,Ans:2
第4000題,Ans:2
第5000題,Ans:2
第6000題,Ans:2
第7000題,Ans:2
第8000題,Ans:2
第9000題,Ans:2
總共66292題
